## test.ipynb: Test the training result and Evaluate model
- tensorflow2 的结果

In [1]:
# Import the necessary libraries
from sklearn.decomposition import PCA
import os
import scipy.io as sio
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras import utils
from sklearn.metrics import classification_report, confusion_matrix
# itertools内置库
import itertools
import spectral
import glob
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Global Variables
windowSize = 5
numPCAcomponents = 30
testRatio = 0.80

# show current path
PATH = os.getcwd()
print (PATH)

e:\Eric_HSI\excise3


In [ ]:
# load the model architecture and weights
model = load_model('./model/HSI_model.h5')

In [3]:
# Define the neccesary functions for later use
# load the Indian pines dataset which is the .mat format
#  load the Indian pines dataset which is the .mat format
def loadIndianPinesData():
    data_path = 'E:\Eric_HSI\hyperspectral_datasets'
    data = sio.loadmat(os.path.join(data_path, 
                      'Indian_pines_corrected.mat'))['data']
    labels = sio.loadmat(os.path.join(data_path, 
                        'Indian_pines_gt.mat'))['groundT']
    
    return data, labels

In [4]:
data_dir= "./predata"
data_root = glob.glob(data_dir + '/*')
# print(data_root)

In [5]:
for name in glob.glob(data_dir + '/*'):
    print(name)

./predata\readme.txt
./predata\X_All_WS_5_PCA_30_testRatio_0.8.npy
./predata\X_test_WS_5_PCA_30_testRatio_0.8.npy
./predata\X_train_WS_5_PCA_30_testRatio_0.8.npy
./predata\y_All_WS_5_PCA_30_testRatio_0.8.npy
./predata\y_test_WS_5_PCA_30_testRatio_0.8.npy
./predata\y_train_WS_5_PCA_30_testRatio_0.8.npy


In [6]:
X_test = np.load(data_root[2])
y_test = np.load(data_root[5])
y_test = utils.to_categorical(y_test)

In [7]:
X_all = np.load(data_root[1])

In [8]:
# apply PCA preprocessing for data sets
def applyPCA(X, numComponents=75):
    newX = np.reshape(X, (-1, X.shape[2]))
    pca = PCA(n_components=numComponents, whiten=True)
    newX = pca.fit_transform(newX)
    newX = np.reshape(newX, (X.shape[0],X.shape[1], numComponents))
    return newX, pca

# slice(start, stop[, step]) 函数实现切片对象，主要用在切片操作函数里的参数传递。
def Patch(data,height_index,width_index):
    #transpose_array = data.transpose((2,0,1))
    #print transpose_array.shape
    height_slice = slice(height_index, height_index+PATCH_SIZE)     # 
    width_slice = slice(width_index, width_index+PATCH_SIZE)
    patch = data[height_slice, width_slice, :]
    
    return patch

In [9]:
# load the original image
X, y = loadIndianPinesData()
# X, y = loadHSIData()
X, pca = applyPCA(X, numComponents=numPCAcomponents)

In [10]:
height = y.shape[0]
width = y.shape[1]
PATCH_SIZE = 5
numComponents = 30
height, width

(145, 145)

In [1]:
image_patch=Patch(X,100,100)
X_test_image = image_patch.reshape(1,image_patch.shape[2],image_patch.shape[0],image_patch.shape[1], 1).astype('float32')
# X_test_image

NameError: name 'Patch' is not defined

In [16]:
# calculate the predicted image
# 因为经过 padding 和 patch 操作，预测过程有一点曲折
outputs = np.zeros((height,width))              # 145*145 ?????????  # 边缘是0，没有计算
for i in range(height-PATCH_SIZE+1):            # 0 - 140
    for j in range(width-PATCH_SIZE+1):        # 0 - 140
        # print(i)
        # print(j)
        p = int(PATCH_SIZE/2)          # 2
        # print(y[i+p][j+p])
        # target = int(y[i+PATCH_SIZE/2, j+PATCH_SIZE/2])
        target = y[i+p][j+p]             # y[2-142]
        if target == 0:
            continue
        else:
            image_patch=Patch(X,i,j)   # 创建 patch 的逆操作，切片
            # print (image_patch.shape)   # 5*5*30
            X_test_image = image_patch.reshape(1,image_patch.shape[0],image_patch.shape[1],image_patch.shape[2]).astype('float32')  # 1*5*5*30    
            # print(X_test_image.shape)
            # prediction = (model.predict_classes(X_test_image))
            prediction = np.argmax(model.predict(X_test_image), axis=-1)
            print(prediction)
            outputs[i+p][j+p] = prediction+1

[2]
[2]
[2]
[2]
[11]
[2]
[2]
[2]
[1]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[11]
[11]
[11]
[14]
[14]
[14]
[14]
[14]
[14]
[5]
[5]


KeyboardInterrupt: 

In [ ]:
ground_truth = spectral.imshow(classes=y, figsize=(6, 6))

In [ ]:
predict_image = spectral.imshow(classes=outputs.astype(int), figsize=(6, 6))